In [1]:
# You should install minerva before running this script

#!pip install git+https://github.com/discovery-unicamp/Minerva-Dev.git
#!pip install kaleido

In [1]:
from minerva.models.nets.time_series.gans import GAN, TTSGAN_Encoder, TTSGAN_Discriminator, TTSGAN_Generator
from minerva.data.data_module_tools import RandomDataModule
import torch
import lightning as L

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:55: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
datamodule = RandomDataModule(
    data_shape = (6,60),
    num_classes = 6,
    num_train_samples = 8,
    num_val_samples = 8,
    num_test_samples= 8,
    batch_size = 4,
)

In [4]:
datamodule.setup("fit")
train_dataloader = datamodule.train_dataloader()

for x, y in train_dataloader:
    print(x.shape, y.shape)
    break

torch.Size([4, 6, 60]) torch.Size([4])


In [5]:
generator = TTSGAN_Generator(seq_len = 60, channels = 6)
discriminator = TTSGAN_Discriminator(seq_len = 60, channels = 6)

model = GAN(generator = generator,
            discriminator = discriminator, 
            loss_gen = torch.nn.MSELoss(),
            loss_dis = torch.nn.MSELoss(),
            )
model

GAN(
  (gen): TTSGAN_Generator(
    (l1): Linear(in_features=100, out_features=600, bias=True)
    (blocks): Gen_TransformerEncoder(
      (0): Gen_TransformerEncoderBlock(
        (0): ResidualAdd(
          (fn): Sequential(
            (0): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
            (1): MultiHeadAttention(
              (keys): Linear(in_features=10, out_features=10, bias=True)
              (queries): Linear(in_features=10, out_features=10, bias=True)
              (values): Linear(in_features=10, out_features=10, bias=True)
              (att_drop): Dropout(p=0.5, inplace=False)
              (projection): Linear(in_features=10, out_features=10, bias=True)
            )
            (2): Dropout(p=0.5, inplace=False)
          )
        )
        (1): ResidualAdd(
          (fn): Sequential(
            (0): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
            (1): FeedForwardBlock(
              (0): Linear(in_features=10, out_features=40, bias=T

In [6]:
'''from lightning.pytorch.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint()
'''

'from lightning.pytorch.callbacks import ModelCheckpoint\n\ncheckpoint_callback = ModelCheckpoint()\n'

In [7]:
from lightning.pytorch.loggers.csv_logs import CSVLogger

logger = CSVLogger(save_dir='.', name='ttsgan', version = 0)

In [8]:
trainer = L.Trainer(accelerator='cpu', devices=1, callbacks=[], logger=logger, max_epochs=1000)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [9]:
trainer.fit(model = model, datamodule = datamodule)

/usr/local/lib/python3.10/dist-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory ./ttsgan/version_0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory ./ttsgan/version_0/checkpoints exists and is not empty.

  | Name     | Type                 | Params | Mode 
----------------------------------------------------------
0 | gen      | TTSGAN_Generator     | 65.3 K | train
1 | dis      | TTSGAN_Discriminator | 97.0 K | train
2 | loss_gen | MSELoss              | 0      | train
3 | loss_dis | MSELoss              | 0      | train
----------------------------------------------------------
162 K     Trainable params
0         Non-trainable params
162 K     Total params
0.649     Total estimated model params size (MB)
138       Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=5` in the `DataLoader` to improve performance.


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=5` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 4:  50%|█████     | 1/2 [00:03<00:03,  0.28it/s, v_num=0]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
ckp = torch.load(f='ttsgan/version_0/checkpoints/epoch=999-step=4000.ckpt')
generator = TTSGAN_Generator(seq_len = 60, channels = 6)
discriminator = TTSGAN_Discriminator(seq_len = 60, channels = 6)

test_gan = GAN(generator = generator,
            discriminator = discriminator, 
            loss_gen = torch.nn.MSELoss(),
            loss_dis = torch.nn.MSELoss(),
            )
model.load_state_dict(ckp['state_dict'])

/tmp/ipykernel_56794/4278499015.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckp = torch.load(f='ttsgan/version_0/checkpoints/epoch=999-step=4000.ckpt')


<All keys matched successfully>